In [4]:
import os
from collections import defaultdict

def contar_palavras(texto):
    return len(texto.split())

def listar_notas_markdown_organizadas(pasta_raiz):
    notas_por_pasta = defaultdict(list)
    contagem_palavras_por_pasta = defaultdict(int)
    total_palavras_geral = 0

    for raiz, _, arquivos in os.walk(pasta_raiz):
        caminho_relativo = os.path.relpath(raiz, pasta_raiz)
        for arquivo in arquivos:
            if arquivo.endswith(".md"):
                caminho_completo = os.path.join(raiz, arquivo)
                nome_nota = os.path.splitext(arquivo)[0]
                
                with open(caminho_completo, 'r', encoding='utf-8') as f:
                    conteudo = f.read()
                    palavras = contar_palavras(conteudo)
                    total_palavras_geral += palavras
                    contagem_palavras_por_pasta[caminho_relativo] += palavras
                    notas_por_pasta[caminho_relativo].append((nome_nota, palavras))

    return notas_por_pasta, contagem_palavras_por_pasta, total_palavras_geral

def salvar_em_markdown(notas_organizadas, contagem_por_pasta, total_geral, caminho_saida):
    with open(caminho_saida, 'w', encoding='utf-8') as f:
        f.write(f"# Índice de Notas Markdown\n\n")
        for pasta, notas in sorted(notas_organizadas.items()):
            titulo_pasta = pasta if pasta != '.' else '[raiz]'
            f.write(f"### {titulo_pasta} — {len(notas)} notas\n")
            for nome_nota, palavras in sorted(notas):
                f.write(f"- [{nome_nota}] — {palavras} palavras\n")
            f.write(f"\n**Total nesta pasta**: {contagem_por_pasta[pasta]} palavras\n\n")

        f.write("---\n")
        f.write(f"## 📊 Total geral: {total_geral} palavras\n")

# Caminhos
caminho_da_pasta = r"C:\Users\nonak\Documents\Thoughts"
caminho_arquivo_saida = os.path.join(caminho_da_pasta, "_index_notas.md")

# Execução
notas_organizadas, contagem_por_pasta, total_palavras = listar_notas_markdown_organizadas(caminho_da_pasta)
salvar_em_markdown(notas_organizadas, contagem_por_pasta, total_palavras, caminho_arquivo_saida)

print(f"Arquivo salvo em: {caminho_arquivo_saida}")


Arquivo salvo em: C:\Users\nonak\Documents\Thoughts\_index_notas.md
